In [1]:
!pip install libauc

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 881.9 MB 2.2 kB/s  eta 0:00:012 |                                | 276 kB 1.1 MB/s eta 0:12:53     |█▍                              | 38.3 MB 1.1 MB/s eta 0:12:20     |█▍                              | 39.8 MB 1.1 MB/s eta 0:12:18     |█████▎                          | 144.4 MB 57.5 MB/s eta 0:00:13     |█████▎                          | 145.2 MB 57.5 MB/s eta 0:00:13     |██████▎                         | 172.8 MB 1.3 MB/s eta 0:09:025319     |████████▋                       | 237.7 MB 31.9 MB/s eta 0:00:21��█▋                   | 348.2 MB 120.5 MB/s eta 0:00:05        | 380.4 MB 28.1 MB/s eta 0:00:18�█████████▉                 | 408.0 MB 35.3 MB/s eta 0:00:14�██████████                 | 410.9 MB 35.3 MB/s eta 0:00:14/s eta 0:00:24     |███████████████▉                | 435.1 MB 19.5 MB/s eta 0:00:23��▏               | 446.3 MB 19.5 MB/s eta 0:00:23��▊               |

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 23.3 MB 113 kB/s eta 0:00:01    |█                               | 727 kB 3.8 MB/s eta 0:00:06     |█████▌                          | 4.0 MB 3.8 MB/s eta 0:00:06
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
from libauc.models import DenseNet121, DenseNet169
from libauc.datasets import CheXpert

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

In [1]:
import dataset

In [5]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
# dataloader
root = './CheXpert/CheXpert-v1.0-small/'
# Index: -1 denotes multi-label mode including 5 diseases
# Returns image with label
traindSet = 
testSet =  CheXpert(csv_path=root+'valid.csv',  image_root_path=root, use_upsampling=False, use_frontal=True, image_size=224, mode='valid', class_index=-1)
trainloader =  torch.utils.data.DataLoader(traindSet, batch_size=32, num_workers=2, shuffle=True)
testloader =  torch.utils.data.DataLoader(testSet, batch_size=32, num_workers=2, shuffle=False)

# paramaters
SEED = 123
BATCH_SIZE = 32
lr = 1e-4
weight_decay = 1e-5

# model
set_all_seeds(SEED)
model = DenseNet121(pretrained=True, last_activation=None, activations='relu', num_classes=5)
model = model.cuda()

# define loss & optimizer
CELoss = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# training
best_val_auc = 0 
for epoch in range(1):
    for idx, data in enumerate(trainloader):
        train_data, train_labels = data
        train_data, train_labels  = train_data.cuda(), train_labels.cuda()
        y_pred = model(train_data)
        loss = CELoss(y_pred, train_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # validation  
        if idx % 400 == 0:
            model.eval()
            with torch.no_grad():    
                test_pred = []
                test_true = [] 
                for jdx, data in enumerate(testloader):
                    test_data, test_labels = data
                    test_data = test_data.cuda()
                    y_pred = model(test_data)
                    test_pred.append(y_pred.cpu().detach().numpy())
                    test_true.append(test_labels.numpy())
            
                test_true = np.concatenate(test_true)
                test_pred = np.concatenate(test_pred)
                val_auc_mean =  roc_auc_score(test_true, test_pred) 
                model.train()

                if best_val_auc < val_auc_mean:
                    best_val_auc = val_auc_mean
                    torch.save(model.state_dict(), 'ce_pretrained_model.pth')

                print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f'%(epoch, idx, val_auc_mean, best_val_auc ))